In [1]:
import pandas as pd
import copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!pip install xgboost

In [4]:
!pip install lightgbm

In [5]:
data = pd.read_csv('train.csv', encoding='CP949')
data.head()

,custid,gender,총구매액,구매건수,평균구매가격,평균할부개월수,구매브랜드종류,내점일수,수입상품_구매비율,주말방문비율,가을_구매건수,겨울_구매건수,봄_구매건수,여름_구매건수,아침_구매건수,저녁_구매건수,점심_구매건수,주구매코너
0,18,0,680100,15,45340,1.7,9,10,26.7,100.0,NaN,9.0,6.0,NaN,NaN,6.0,9.0,화장품
1,21,0,353450,9,39272,1.2,6,8,11.1,37.5,NaN,5.0,4.0,NaN,3.0,NaN,6.0,영캐주얼
2,23,0,5671400,36,157539,2.8,22,16,5.6,37.5,7.0,17.0,12.0,NaN,10.0,11.0,15.0,장신구
3,26,0,1964000,28,70143,1.4,15,14,39.3,28.6,5.0,8.0,15.0,NaN,NaN,18.0,10.0,화장품
4,35,0,885000,5,177000,6.0,5,2,0.0,100.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,피혁A


In [6]:
dataP = data.copy()

In [7]:
obj=['주구매코너'] 

In [8]:
dataP[obj] = dataP[obj].apply(lambda x: x.astype('category').cat.codes) 

In [9]:
con = ['가을_구매건수','겨울_구매건수','봄_구매건수','여름_구매건수','아침_구매건수','저녁_구매건수','점심_구매건수' ]

In [10]:
from sklearn.impute import SimpleImputer
imputer_con = SimpleImputer(strategy="most_frequent")  
imputer_con.fit(dataP[con])

SimpleImputer(strategy='most_frequent')

In [11]:
x = imputer_con.transform(dataP[con])
x

array([[ 1.,  9.,  6., ...,  1.,  6.,  9.],
       [ 1.,  5.,  4., ...,  3.,  1.,  6.],
       [ 7., 17., 12., ..., 10., 11., 15.],
       ...,
       [12.,  6.,  6., ..., 16.,  1., 21.],
       [ 6.,  4.,  2., ...,  1.,  2., 11.],
       [ 6.,  5., 10., ...,  8.,  5., 19.]])

In [12]:
dataP[con] = x

In [13]:
dfX = dataP.drop(['custid','gender'], axis=1) 
dfy = dataP['gender']

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#model = KNeighborsClassifier()
model = LogisticRegression()
#model = DecisionTreeClassifier()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [15]:
scores = cross_val_score(model, dfX, dfy); scores

array([0.696, 0.697, 0.695, 0.697, 0.695])

In [16]:
print("Mean: {:.3f}\nStd: {:.3f}\nMin: {:.3f}\nMax: {:.3f}".format(
    scores.mean(), scores.std(), scores.min(), scores.max()))
# 분산이 작으면 Overfitting 이 적다

Mean: 0.696
Std: 0.001
Min: 0.695
Max: 0.697


In [17]:
!pip install imblearn

In [18]:
from imblearn.combine import *
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [19]:
scores = cross_val_score(model, dfX, dfy); scores

array([0.696, 0.697, 0.695, 0.697, 0.695])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.25, random_state=0)

In [ ]:
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

In [30]:
XX, yy = SMOTETomek(random_state=0).fit_sample(X_train, y_train)

In [31]:
tree3 = DecisionTreeClassifier(max_depth=4, random_state=0)
tree3.fit(XX, yy)
y_pred3 = tree3.predict(X_test)

print(classification_report(y_test, y_pred3))
# (precision) 0 이라고 예측한 데이터의 74%만 실제로 0
# (precision) 1 이라고 예측한 데이터의 33%만 실제로 1
# (recall) 실제 0인 데이터중 56%만 0으로 판별됨 
# (recall) 실제 1인 데이타중 52%만 1로 판별됨

              precision    recall  f1-score   support

           0       0.74      0.56      0.64       883
           1       0.33      0.52      0.41       367

    accuracy                           0.55      1250
   macro avg       0.54      0.54      0.52      1250
weighted avg       0.62      0.55      0.57      1250



In [32]:
from sklearn.dummy import DummyClassifier
print(y_test.value_counts())
DummyClassifier(strategy='most_frequent').fit(X_train, y_train).score(X_test, y_test)

0    883
1    367
Name: gender, dtype: int64


0.7064

In [33]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import plot_importance

In [34]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

In [35]:
params = {'max_depth':3,
          'eta':0.1,
          'objective':'binary:logistic',
          'eval_metric':'logloss',
          'early_stoppings':100
         }
num_rounds = 400

In [36]:
# train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval'로 명기
wlist = [(dtrain, 'train'),(dtest, 'eval')]
# 하이퍼 파라미터와 early stoppinig 파라미터를 train() 함수의 파라미터로 전달
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)

[15:05:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.67505	eval-logloss:0.67724
[1]	train-logloss:0.66029	eval-logloss:0.66433
[2]	train-logloss:0.64789	eval-logloss:0.65309
[3]	train-logloss:0.63754	eval-logloss:0.64475
[4]	train-logloss:0.62892	eval-logloss:0.63700
[5]	train-logloss:0.62165	eval-logloss:0.63109
[6]	train-logloss:0.61543	eval-logloss:0.62605
[7]	train-logloss:0.61015	eval-logloss:0.62206
[8]	train-logloss:0.60568	eval-logloss:0.61911
[9]	train-logloss:0.60181	eval-logloss:0.61679
[10]	train-logloss:0.59833	eval-logloss:0.61454
[11]	train-logloss:0.59566	eval-logloss:0.61296
[12]	train-logloss:0.59313	eval-logloss:0

[162]	train-logloss:0.50162	eval-logloss:0.60652
[163]	train-logloss:0.50114	eval-logloss:0.60650
[164]	train-logloss:0.50081	eval-logloss:0.60659
[165]	train-logloss:0.50036	eval-logloss:0.60648
[166]	train-logloss:0.50027	eval-logloss:0.60658
[167]	train-logloss:0.49972	eval-logloss:0.60645
[168]	train-logloss:0.49956	eval-logloss:0.60645
[169]	train-logloss:0.49897	eval-logloss:0.60664
[170]	train-logloss:0.49866	eval-logloss:0.60647
[171]	train-logloss:0.49806	eval-logloss:0.60619
[172]	train-logloss:0.49759	eval-logloss:0.60631
[173]	train-logloss:0.49721	eval-logloss:0.60650
[174]	train-logloss:0.49665	eval-logloss:0.60644
[175]	train-logloss:0.49635	eval-logloss:0.60655
[176]	train-logloss:0.49601	eval-logloss:0.60646
[177]	train-logloss:0.49582	eval-logloss:0.60652
[178]	train-logloss:0.49552	eval-logloss:0.60657
[179]	train-logloss:0.49506	eval-logloss:0.60660
[180]	train-logloss:0.49490	eval-logloss:0.60636
[181]	train-logloss:0.49443	eval-logloss:0.60650
[182]	train-logloss:

[330]	train-logloss:0.44251	eval-logloss:0.61819
[331]	train-logloss:0.44225	eval-logloss:0.61826
[332]	train-logloss:0.44173	eval-logloss:0.61854
[333]	train-logloss:0.44140	eval-logloss:0.61839
[334]	train-logloss:0.44133	eval-logloss:0.61847
[335]	train-logloss:0.44126	eval-logloss:0.61859
[336]	train-logloss:0.44118	eval-logloss:0.61855
[337]	train-logloss:0.44096	eval-logloss:0.61872
[338]	train-logloss:0.44093	eval-logloss:0.61864
[339]	train-logloss:0.44088	eval-logloss:0.61876
[340]	train-logloss:0.44068	eval-logloss:0.61862
[341]	train-logloss:0.44035	eval-logloss:0.61866
[342]	train-logloss:0.44003	eval-logloss:0.61894
[343]	train-logloss:0.43956	eval-logloss:0.61898
[344]	train-logloss:0.43946	eval-logloss:0.61912
[345]	train-logloss:0.43942	eval-logloss:0.61923
[346]	train-logloss:0.43900	eval-logloss:0.61925
[347]	train-logloss:0.43861	eval-logloss:0.61919
[348]	train-logloss:0.43838	eval-logloss:0.61951
[349]	train-logloss:0.43798	eval-logloss:0.61975
[350]	train-logloss:

In [37]:
pred_probs = xgb_model.predict(dtest)
print('predict() 수행 결과값을 10개만 표시, 예측 확률값으로 표시됨')
print(np.round(pred_probs[:10], 3))

preds = [1 if x > 0.5 else 0 for x in pred_probs]

predict() 수행 결과값을 10개만 표시, 예측 확률값으로 표시됨
[0.273 0.453 0.275 0.449 0.191 0.158 0.161 0.193 0.18  0.325]


In [38]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [39]:
get_clf_eval(y_test, preds)

오차행렬
[[787  96]
 [315  52]]
정확도: 0.6712, 정밀도: 0.3514, 재현율: 0.1417


In [40]:
XX.columns

Index(['총구매액', '구매건수', '평균구매가격', '평균할부개월수', '구매브랜드종류', '내점일수', '수입상품_구매비율',
       '주말방문비율', '가을_구매건수', '겨울_구매건수', '봄_구매건수', '여름_구매건수', '아침_구매건수',
       '저녁_구매건수', '점심_구매건수', '주구매코너'],
      dtype='object')

In [41]:
col = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

In [42]:
XX.head()

,총구매액,구매건수,평균구매가격,평균할부개월수,구매브랜드종류,내점일수,수입상품_구매비율,주말방문비율,가을_구매건수,겨울_구매건수,봄_구매건수,여름_구매건수,아침_구매건수,저녁_구매건수,점심_구매건수,주구매코너
0,404285,3,134762,2.3,3,2,33.3,0.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,22
1,2407157,32,75224,1.7,22,17,9.4,52.9,3.0,3.0,6.0,20.0,3.0,8.0,21.0,14
2,4664283,78,59798,1.8,24,22,1.3,45.5,17.0,12.0,29.0,20.0,1.0,23.0,55.0,14
3,3299200,11,299927,1.9,9,3,45.5,33.3,1.0,1.0,11.0,1.0,1.0,1.0,11.0,16
4,555480,5,111096,2.6,4,4,20.0,75.0,2.0,1.0,1.0,3.0,1.0,3.0,2.0,17


In [43]:
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(nestimators=400)
evals = [(X_test, y_test)]
lgbm_wrapper.fit(XX, yy, feature_name=col, early_stopping_rounds=100, 
                 eval_metric="logloss", eval_set=evals, verbose=True)
pred = lgbm_wrapper.predict(X_test)

[LightGBM] [Warning] Unknown parameter: nestimators
[1]	valid_0's binary_logloss: 0.684352
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.678241
[3]	valid_0's binary_logloss: 0.672711
[4]	valid_0's binary_logloss: 0.666675
[5]	valid_0's binary_logloss: 0.662707
[6]	valid_0's binary_logloss: 0.66007
[7]	valid_0's binary_logloss: 0.657454
[8]	valid_0's binary_logloss: 0.656766
[9]	valid_0's binary_logloss: 0.653329
[10]	valid_0's binary_logloss: 0.651607
[11]	valid_0's binary_logloss: 0.651391
[12]	valid_0's binary_logloss: 0.649682
[13]	valid_0's binary_logloss: 0.649541
[14]	valid_0's binary_logloss: 0.647985
[15]	valid_0's binary_logloss: 0.647646
[16]	valid_0's binary_logloss: 0.64678
[17]	valid_0's binary_logloss: 0.645509
[18]	valid_0's binary_logloss: 0.64361
[19]	valid_0's binary_logloss: 0.643435
[20]	valid_0's binary_logloss: 0.64084
[21]	valid_0's binary_logloss: 0.641548
[22]	valid_0's binary_logloss: 0.64204
[23]	valid_0's binar